## Chapter 7
# Gaussian Elimination

## 7.7 Lab: Threshold Secret-Sharing

_Consult the book for a full description of the secret-sharing scheme._

### 7.7.3 Implementing the scheme

In [1]:
import sys
sys.path.append('../')
from vecutil import list2vec
from GF2 import one

In [2]:
a0 = list2vec([one, one, 0, one, 0, one])
b0 = list2vec([one, one, 0, 0,   0, one])

### 7.7.4 Generating $\boldsymbol{u}$

**Task 7.7.1:** Write a procedure `choose_secret_vector(s, t)` with the following spec:

* _input:_ $GF(2)$ field elements `s` and `t` (i.e. bits)
* _output:_ a random 6-vector $\boldsymbol{u}$ such that $\boldsymbol{a}_0 \cdot \boldsymbol{u} = s$ and $\boldsymbol{b}_0 \cdot \boldsymbol{u} = t$

In [3]:
import random

def random_gf2_vector(length):
    return list2vec([random.randint(0,1) * one for _ in range(length)])

def choose_secret_vector(s, t):
    while True:
        u = random_gf2_vector(len(a0.D))
        if a0 * u == s and b0 * u == t:
            return u

In [4]:
u = choose_secret_vector(0, one)

### 7.7.5 Finding vectors that satisfy the requirement

**Task 7.7.2:** Select vectors $\boldsymbol{a}_1, \boldsymbol{b}_1, \boldsymbol{a}_2, \boldsymbol{b}_2, \boldsymbol{a}_3, \boldsymbol{b}_3, \boldsymbol{a}_4, \boldsymbol{b}_4$ over $GF(2)$ so that the requirement is satisfied: _For any three pairs, the corresponding six vectors are linearly independent_.

Include in your answer any code you used, and the vectors you came up with.
**_Hint:_** Try selecting eight random vectors and testing whether they satisfy the requirement. Repeat until you succeed. Use the `independence` module.

In [5]:
from independence import is_independent
from itertools import combinations

# includes given a0 and b0 in results
def find_independent_gf2_vectors(n, a0, b0):
    while True:
        vectors = [random_gf2_vector(len(a0.D)) for _ in range(n)]
        vectors = [a0, b0] + vectors
        all_combo_indices = combinations(range(n // 2 + 1), 3)
        if all(is_independent(
            [vectors[i*2],vectors[i*2+1],vectors[j*2],vectors[j*2+1],vectors[k*2],vectors[k*2+1]]
        ) for i, j, k in all_combo_indices):
            return vectors

In [6]:
independent_gf2_vectors = find_independent_gf2_vectors(8, a0, b0)

In [7]:
independent_gf2_vectors

[Vec({0, 1, 2, 3, 4, 5},{0: one, 1: one, 2: 0, 3: one, 4: 0, 5: one}),
 Vec({0, 1, 2, 3, 4, 5},{0: one, 1: one, 2: 0, 3: 0, 4: 0, 5: one}),
 Vec({0, 1, 2, 3, 4, 5},{0: 0, 1: 0, 2: 0, 3: one, 4: 0, 5: one}),
 Vec({0, 1, 2, 3, 4, 5},{0: one, 1: 0, 2: one, 3: 0, 4: one, 5: one}),
 Vec({0, 1, 2, 3, 4, 5},{0: one, 1: 0, 2: one, 3: 0, 4: 0, 5: one}),
 Vec({0, 1, 2, 3, 4, 5},{0: 0, 1: one, 2: 0, 3: 0, 4: 0, 5: 0}),
 Vec({0, 1, 2, 3, 4, 5},{0: 0, 1: 0, 2: 0, 3: one, 4: one, 5: 0}),
 Vec({0, 1, 2, 3, 4, 5},{0: 0, 1: 0, 2: one, 3: one, 4: one, 5: one}),
 Vec({0, 1, 2, 3, 4, 5},{0: 0, 1: 0, 2: one, 3: one, 4: one, 5: 0}),
 Vec({0, 1, 2, 3, 4, 5},{0: 0, 1: one, 2: one, 3: one, 4: 0, 5: one})]

### 7.7.6 Sharing a string

Now that we can share two bits, we can share an arbitrarily long string. The module `bitutil` defines procedures `str2bits(str)`, `bits2str(bitlist)`, `bits2mat(bitlist, nrows)` and `mat2bits(M)`.

You can use `str2bits` to transform a string, say `"Rosebud"`, into a list of bits, and use `bits2mat` to transform the list of bits to a $2 \times n$ matrix.

For each column of this matrix, you can use the procedure `choose_secret_vector(s, t)` of Task 7.7.1 to obtain a corresponding secret vector $\boldsymbol{u}$, constructing a matrix $U$ whose columns are the secret vectors.

To compute the shares of the TSs, multiply the matrix

$\begin{bmatrix}\boldsymbol{a}_0\\\boldsymbol{b}_0\\\boldsymbol{a}_1\\\boldsymbol{b}_1\\\boldsymbol{a}_2\\\boldsymbol{b}_2\\\boldsymbol{a}_3\\\boldsymbol{b}_3\\\boldsymbol{a}_4\\\boldsymbol{b}_4\end{bmatrix}$

times $U$. The third and fourth rows of the product form the share for TA 1, and so on.

In [8]:
from bitutil import str2bits, bits2mat

bits = str2bits("Rosebud")
mat = bits2mat(bits)

from matutil import listlist2mat, mat2coldict, coldict2mat, rowdict2mat, mat2rowdict

mat_columns = mat2coldict(mat).values()
U = coldict2mat([choose_secret_vector(column[0], column[1]) for column in mat_columns])
shares_mat = rowdict2mat(independent_gf2_vectors) * U

In [9]:
print(shares_mat)


         0   1  10  11  12  13   2   3   4   5   6   7   8   9
     ---------------------------------------------------------
 0  |    0 one one one   0   0 one   0 one one one   0   0   0
 1  |  one   0   0 one   0 one one one one one   0 one one one
 2  |  one   0   0 one   0 one   0 one   0   0   0   0   0 one
 3  |  one   0 one   0 one one   0 one   0 one   0   0 one   0
 4  |  one   0 one   0 one one one one one one   0 one   0 one
 5  |  one   0   0 one   0 one   0   0 one one one one one   0
 6  |  one one one   0   0 one one one one   0 one   0   0   0
 7  |    0   0 one one one   0 one one   0 one one   0 one   0
 8  |    0 one   0   0 one   0 one one   0 one   0 one   0   0
 9  |  one   0 one   0 one one   0 one   0   0   0   0 one one



In [10]:
from solver import solve

#recovered_u = solve(rowdict2mat(independent_gf2_vectors[2:8]), rowdict2mat(list(mat2rowdict(shares_mat).values())[2:8]))

## 7.8 Lab: Factoring integers

### 7.8.1 First attempt to use square roots

**Task 7.8.1:** To find integers $a$ and $b$ such that $a^2 - b^2 = N$, write a procedure `root_method(N)` to implement the following algorithm:

* Initialize integer $a$ to be an integer greater than $\sqrt{N}$
* Check if $\sqrt{a^2 - N}$ is an integer.
* If so, let $b = \sqrt{a^2 - N}$. Success! Return $a - b$.
* If not, repeat with the next greater value of $a$.

The module `factoring_support` provides a procedure `intsqrt(x)` with the following spec:
* _input:_ an integer $x$
* _output:_ an integer $y$ such that $y * y$ is close to $x$ and, if $x$ happens to be a perfect square, $y * y$ is exactly $x$.

You should use `intsqrt(x)` in your implementation of the above algorithm. Try it out with 55, 77, 156771 and 118. 

In [11]:
from factoring_support import intsqrt

def root_method(N):
    a = intsqrt(N) + 1
    while a < N ** 2: # avoid running forever, against the book's instructions
        x = a ** 2 - N
        b = intsqrt(x)
        if b * b == x:
            return a - b
        else:
            a += 1
    return None

In [12]:
root_method(55)

5

In [13]:
root_method(77)

1

In [14]:
root_method(156771)

9

In [15]:
root_method(118)

### 7.8.2 Euclid's algorithm for greatest common divisor

**Task 7.8.2:** Enter the code for `gcd` and try it out. Use `randint(a,b)` to generate some very big integers $r$, $s$, $t$. Then set $a = r * s$ and $b = s * t$, and find the greatest common divisor $d$ of $a$ and $b$. Verify that $d$ has the following properties:

* $a$ is divisible by $d$
* $b$ is divisible by $d$, and
* $d \geq s$

In [16]:
def gcd(x, y):
    return x if y == 0 else gcd(y, x % y)

In [17]:
from random import randint

for _ in range(100):
    r = randint(10_000, 10_000_000)
    s = randint(10_000, 10_000_000)
    t = randint(10_000, 10_000_000)
    a = r * s
    b = s * t
    
    d = gcd(a, b)
    assert(a % d == 0)
    assert(b % d == 0)
    assert(d >= s)

### 7.8.3 Using square roots revisited

**Task 7.8.3:** Let $N = 367160330145890434494322103$, let $a = 67469780066325164$, and let $b = 9429601150488992$, and verity that $a * a - b * b$ is divisible by $N$. That means that the greatest common divisor of $a - b$ and $N$ has a chance of being a nontrivial divisor of $N$. Test this using the `gcd` procedure, and report the nontrivial divisor you find.

In [18]:
N=367160330145890434494322103 
a=67469780066325164
b=9429601150488992

assert((a * a - b * b) % N == 0)

In [19]:
gcd(a - b, N)

19117318483477

The module `factoring_support` defines a procedure `dumb_factor(x, primeset)` with the following spec:
* _input:_ an integer `x` and a set `primeset` of primes
* _output:_ if there are primes $p_1, ..., p_s$ in `primeset` and positive integers $e_1, e_2, ..., e_s$ (the exponents) such that $x = p_1^{e_1}p_2^{e_2} \cdots p_s^{e_s}$, then the procedure returns the list $[(p_1,e_1),(p_2,e_2),...,(p_s,e_s)]$ of pairs (prime, exponent). If not, the procedure returns the empty list.

Here are some examples:
```
>>> dumb_factor(75, {2,3,5,7})
[(3, 1), (5, 2)]
>>> dumb_factor(30, {2,3,5,7})
[(2, 1), (3, 1), (5, 1)]
>>> dumb_factor(1176, {2,3,5,7})
[(2, 3), (3, 1), (7, 2)]
>>> dumb_factor(2 * 17, {2,3,5,7})
[]
>>> dumb_factor(2 * 3 * 5 * 19, {2,3,5,7})
[]
```

**Task 7.8.4:** Define `primeset={2,3,5,7,11,13}`. Try out `dumb_factor(x, primeset)` on integers `x = 12`, `x = 154`, `x = 2 * 3 * 3 * 3 * 11 * 11 * 13`, `x = 2 * 17`, `x = 2 * 3 * 5 * 7 * 19`.  Report the results.

In [20]:
from factoring_support import dumb_factor

primeset={2,3,5,7,11,13}

for x in [12, 154, 2*3*3*3*11*11*13, 2*17, 2*3*5*7*19]:
    print('x: ', x)
    print('dumb_factor(x, primeset): ', dumb_factor(x, primeset))

x:  12
dumb_factor(x, primeset):  [(2, 2), (3, 1)]
x:  154
dumb_factor(x, primeset):  [(2, 1), (7, 1), (11, 1)]
x:  84942
dumb_factor(x, primeset):  [(2, 1), (3, 3), (11, 2), (13, 1)]
x:  34
dumb_factor(x, primeset):  []
x:  3990
dumb_factor(x, primeset):  []


**Task 7.8.5:** From the `GF2` module, import the value `one`. Write a procedure `int2GF2(i)` that, given an integer `i`, returns `one` if `i` is odd and `0` if `i` is even.
```
>>> int2GF2(3)
one
>>> int2GF2(4)
0
```

In [21]:
def int2GF2(i):
    return 0 if i % 2 == 0 else one

In [22]:
int2GF2(3)

one

In [23]:
int2GF2(4)

0

**Task 7.8.6:** From the module `vec`, import `Vec`. Write a procedure `make_Vec(primeset, factors)` with the following spec:
* _input:_ a set of primes `primeset` and a list `factors`$=[(p_1,e_1),(p_2,e_2),...,(p_s,e_s)]$ such as produced by `dum_factor`, where every $p_i$ belongs to `primeset`
* _output:_ a `primeset`-vector $\boldsymbol{v}$ over $GF(2)$ with domain `primeset` such that `v[p_i] = int2GF2(e_i)` for `i = 1, ..., s`

For example,
```
>>> make_Vec({2,3,5,7,11}, [(3,1)])
Vec({3, 2, 11, 5, 7}, {3: one})
>>> make_Vec({2,3,5,7,11}, [(2,17), (3, 0), (5, 1), (11, 3)])
Vec({3, 2, 11, 5, 7}, {11: one, 2: one, 3: 0, 5: one})
```

In [24]:
from vec import Vec

def make_Vec(primeset, factors):
    return Vec(primeset, {p: int2GF2(e) for p, e in factors})

In [26]:
make_Vec({2,3,5,7,11}, [(3,1)])

Vec({2, 3, 5, 7, 11},{3: one})

In [27]:
make_Vec({2,3,5,7,11}, [(2,17), (3, 0), (5, 1), (11, 3)])

Vec({2, 3, 5, 7, 11},{2: one, 3: 0, 5: one, 11: one})

**Task 7.8.7:** Write a procedure `find_candidates(N, primeset)` that, given an integer $N$ to factor and a set `primeset` of primes, finds `len(primeset) + 1` integers `a` for which `a * a - N` can be factored completely over `primeset`. The procedure returns two lists:

* the list `roots` consisting of $a_0, a_1, a_2, ...$ such that $a_i \cdot a_i - N$ can be factored completely over `primeset`, and
* the list `rowlist` such that element $i$ is the `primeset`-vector over $GF(2)$ corresponding to $a_i$ (that is, the vector produced by `make_Vec`).

The algorithm should initialize
```
roots = []
rowlist = []
```
and then iterate for `x = intsqrt(N) + 2, intsqrt(N) + 3, ..., ` and for each value of `x`,
* if `x * x - N` can be factored completely over `primeset`,
  - append `x` to roots,
  - append to `rowlist` the vector corresponding to the factors of `x * x - N`

continuing until at least `len(primeset) + 1` roots and vectors have been accumulated.

Try out your procedure on `N = 2419` by calling `find_candidates(N, primes(32))`.

After the loop completes, the value of `roots` should be the list `[51,52,53,58,61,62,63,67,68,71,77,79]`.  (Consult the book for the full expected value of `rowlist`.

In [28]:
def find_candidates(N, primeset):
    roots = []
    rowlist = []

    x = intsqrt(N) + 2
    while len(roots) < len(primeset) + 1:
        factors = dumb_factor(x * x - N, primeset)
        if len(factors) > 0:
            roots.append(x)
            rowlist.append(make_Vec(primeset, factors))
        x += 1
    return roots, rowlist

In [29]:
from factoring_support import primes
N = 2419
roots, rowlist = find_candidates(N, primes(32))

In [30]:
roots # matches expected value above

[51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79]

In [31]:
rowlist # this does indeed also matches the expected value listed in the book

[Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 7: one, 13: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: one, 5: one, 19: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 5: one, 13: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: one, 5: one, 7: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 7: one, 31: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: one, 5: 0, 19: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 5: 0, 31: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: 0, 5: one, 23: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: 0, 5: one, 7: 0}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 19: one, 23: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 5: one, 13: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 7: 0, 13: one})]

**Task 7.8.8:** To try to find a factor, let $a = 53 \cdot 77$ and let $b = 2 \cdot 3^2 \cdot 5 \cdot 13$, and compute `gcd(a - b, N)`. Did you find a proper divisor of $N$?

In [32]:
a = 53 * 77
b = 2 * 3 **2 * 5 * 13
gcd(a - b, N)

41

In [33]:
N / 41 # It checks out!

59.0

**Task 7.8.9:** To again try to find a factor of $N$ (just for practice). Let $a = 52 \cdot 67 \cdot 71$ and let $b = 2 \cdot 3^2 \cdot 5 \cdot 19 \cdot 23$, and compute `gcd(a - b, N)`. Did you find a proper divisor of $N$?

In [34]:
a = 52 * 67 * 71
b = 2 * 3**2 * 5 * 19 * 23
gcd(a - b, N)

2419

In [35]:
N / 2419 # trivial divisor

1.0

In [36]:
from echelon import transformation_rows

M = transformation_rows(rowlist)
print(rowdict2mat(M))


          0   1  10  11   2   3   4   5   6   7   8   9
      -------------------------------------------------
  0  |  one   0   0   0   0   0   0   0   0   0   0   0
  1  |    0 one   0   0   0   0   0   0   0   0   0   0
 10  |    0   0 one   0 one   0   0   0   0   0   0   0
 11  |    0 one   0 one one   0   0 one   0   0   0   0
  2  |    0   0   0   0   0   0 one   0   0   0   0   0
  3  |    0   0   0   0   0   0 one   0   0 one   0   0
  4  |  one   0   0   0 one   0   0   0   0   0   0   0
  5  |    0 one   0   0   0   0   0 one   0   0   0   0
  6  |  one   0   0   0 one one   0   0   0   0   0   0
  7  |  one one   0   0 one   0 one one one   0   0   0
  8  |    0 one   0   0   0   0   0 one   0   0 one   0
  9  |    0 one   0   0   0   0   0   0   0 one   0 one



In [37]:
print(rowdict2mat(M) * rowdict2mat(rowlist)) # echelon form


        11  13 17  19   2  23 29   3  31   5   7
      ------------------------------------------
  0  |   0 one  0   0 one   0  0   0   0   0 one
  1  |   0   0  0 one   0   0  0 one   0 one   0
 10  |   0   0  0   0   0   0  0   0   0   0   0
 11  |   0   0  0   0   0   0  0   0   0   0   0
  2  |   0   0  0   0 one   0  0 one one   0 one
  3  |   0   0  0   0   0 one  0 one one one one
  4  |   0   0  0   0   0   0  0 one   0 one one
  5  |   0   0  0   0   0   0  0   0   0 one   0
  6  |   0   0  0   0   0   0  0   0   0   0   0
  7  |   0   0  0   0   0   0  0   0   0   0   0
  8  |   0   0  0   0   0   0  0   0   0   0   0
  9  |   0   0  0   0   0   0  0   0   0   0   0



**Task 7.8.10:** Define a procedure `find_a_and_b(v, roots, N)` that, given a vector `v` (one of the rows of `M`, the echelon-transformation matrix), the list `roots`, and the integer `N` to factor, computes a pair $(a, b)$ of integers such that $a^2 - b^2$ is a multiple of `N`. Your procedure should work as follows:

* Let `alist` be the list of elements of `roots` corresponding to nonzero entries of the vector `v` (Use a comprehension.)
* Let `a` be the product of these. (Use the procedure `prod(alist)` defined in the module `factoring_support`.)
* Let `c` be the produce of $\{$ $x \cdot x - N : x \in $ `alist` $\}$
* Let `b` be `intsqrt(c)`.
* Verify using an assertion that `b * b == c`.
* Return the pair `(a, b)`.

Try out your procedure with `v` being the last row of `M`. See if `a - b` and `N` have a nontrivial common divisor. If it doesn't work, try it with `v` being the second-to-last row of `M`, etc.

In [38]:
from factoring_support import prod

def find_a_and_b(v, roots, N):
    alist = [roots[i] for i, val in enumerate(v.f.values()) if val != 0]
    a = prod(alist)
    c = prod([x * x - N for x in alist])
    b = intsqrt(c)
    assert(b * b == c)
    return a, b

In [39]:
a, b = find_a_and_b(M[-1], roots, N)
a, b

(13498888, 778050)

In [40]:
gcd(a - b, N) # only the trivial common divisor. Try with second-to-last row of M (echelon transformation matrix of rowlist)

1

In [41]:
a, b = find_a_and_b(M[-2], roots, N)
gcd(a - b, N) # yup! nontrivial common divisor.

41

**Task 7.8.11:** Let $N = 2461799993978700679$, and try to factor $N$
* Let `primelist` be the set of primes up to 10,000.
* Use `find_candidates(N, primelist)` to compute the lists `roots` and `rowlist`.
* Use `echelon.transformation_rows(rowlist)` to get a matrix `M`.
* Let `v` be the last row of `M`, and find `a` and `b` using `find_a_and_b(v, roots, N)`.
* See if `a - b` has a nontrivial common divisor with `N`. If not, repeat with `v` being the second-to-last row of `M` or the third-to-last row...

Give a nontirvial divisor of $N$.


In [44]:
def factor(N):
    primelist = primes(10_000)
    roots, rowlist = find_candidates(N, primelist)
    M = transformation_rows(rowlist, sorted(primelist, reverse=True))
    for v in reversed(M):
        a, b = find_a_and_b(v, roots, N)
        divisor = gcd(a - b, N)
        if divisor != 1 and divisor != N:
            return divisor

In [45]:
N=2461799993978700679
factor(N)

1230926561

**Task 7.8.12:** Let $N = 20672783502493917028427$, and try to factor $N$. This time, since $N$ is a lot bigger, finding $K + 1$ rows will take a lot longer, perhaps size to ten minutes depending on your computer. Finding $M$ could take a few minutes. (Note that I have already implemented the next task to speed things up at least a little bit).

In [46]:
N=20672783502493917028427
factor(N)

1230926561

**Task 7.8.13:** Here is a way to speed up finding $M$: The procedure `echelon.transformation_rows` takes an optional second argument, a list of column-labels. The list instructs the procedure in which order to handle column-labels. The proceure works much faster if the list consists of the primes of `primelist` in descending order:
```
>>> M_rows = echelon.transformation_rows(rowlist, sorted(primelist, reverse=True))
```